In [2]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.utils.data as Data
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
from collections import OrderedDict

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [4]:
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.1351, -0.0167],
        [-0.4022, -1.0507],
        [ 0.6495,  0.2650],
        [-0.5402,  1.2629],
        [ 0.5883,  0.7129],
        [-1.4981, -1.4955],
        [ 0.0465, -0.1362],
        [-0.4845,  0.7967],
        [-0.4642, -1.2957],
        [ 1.1027, -0.5051]])tensor([ 4.5328,  6.9666,  4.5997, -1.1730,  2.9391,  6.2817,  4.7603,  0.5102,
         7.6759,  8.1226])


In [6]:
net=nn.Sequential(OrderedDict([
    ('linear',nn.Linear(num_inputs,1))
]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)
nn.init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0004,  0.0121]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [16]:
loss = nn.MSELoss()
optimizer =torch.optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net[0].parameters(), 'lr': 0.02}
            ], lr=0.03)
print(optimizer)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
#print(optimizer)



SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [17]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.459637
epoch 2, loss: 0.014882
epoch 3, loss: 0.000620


In [18]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4]Parameter containing:
tensor([[ 1.9963, -3.3892]], requires_grad=True)
4.2Parameter containing:
tensor([4.1899], requires_grad=True)
